In [21]:
%load_ext autoreload
%autoreload 2

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fileh5 = h5py.File("./ksp_data/brain_7t_kspace.h5", 'r')
dataset = fileh5['dataset']
raw_data = dataset['data']
xml = dataset['xml']

In [6]:
raw_data.shape

(23500,)

In [ ]:
one_slice = raw_data[1024]
print(one_slice.dtype) # hard to identify the structure

{'names': ['head', 'traj', 'data'], 'formats': [[('version', '<u2'), ('flags', '<u8'), ('measurement_uid', '<u4'), ('scan_counter', '<u4'), ('acquisition_time_stamp', '<u4'), ('physiology_time_stamp', '<u4', (3,)), ('number_of_samples', '<u2'), ('available_channels', '<u2'), ('active_channels', '<u2'), ('channel_mask', '<u8', (16,)), ('discard_pre', '<u2'), ('discard_post', '<u2'), ('center_sample', '<u2'), ('encoding_space_ref', '<u2'), ('trajectory_dimensions', '<u2'), ('sample_time_us', '<f4'), ('position', '<f4', (3,)), ('read_dir', '<f4', (3,)), ('phase_dir', '<f4', (3,)), ('slice_dir', '<f4', (3,)), ('patient_table_position', '<f4', (3,)), ('idx', [('kspace_encode_step_1', '<u2'), ('kspace_encode_step_2', '<u2'), ('average', '<u2'), ('slice', '<u2'), ('contrast', '<u2'), ('phase', '<u2'), ('repetition', '<u2'), ('set', '<u2'), ('segment', '<u2'), ('user', '<u2', (8,))]), ('user_int', '<i4', (8,)), ('user_float', '<f4', (8,))], 'O', 'O'], 'offsets': [0, 344, 360], 'itemsize': 376}

In [9]:
def summarize_dtype(dtype, indent=0):
    prefix = "  " * indent
    if dtype.fields is None:
        print(f"{prefix}{dtype}")
        return
    for name, (subdtype, offset) in dtype.fields.items():
        print(f"{prefix}{name}: ", end="")
        if subdtype.fields:
            print()  # newline
            summarize_dtype(subdtype, indent + 1)
        else:
            print(subdtype)

In [11]:
summarize_dtype(one_slice.dtype)

head: 
  version: uint16
  flags: uint64
  measurement_uid: uint32
  scan_counter: uint32
  acquisition_time_stamp: uint32
  physiology_time_stamp: ('<u4', (3,))
  number_of_samples: uint16
  available_channels: uint16
  active_channels: uint16
  channel_mask: ('<u8', (16,))
  discard_pre: uint16
  discard_post: uint16
  center_sample: uint16
  encoding_space_ref: uint16
  trajectory_dimensions: uint16
  sample_time_us: float32
  position: ('<f4', (3,))
  read_dir: ('<f4', (3,))
  phase_dir: ('<f4', (3,))
  slice_dir: ('<f4', (3,))
  patient_table_position: ('<f4', (3,))
  idx: 
    kspace_encode_step_1: uint16
    kspace_encode_step_2: uint16
    average: uint16
    slice: uint16
    contrast: uint16
    phase: uint16
    repetition: uint16
    set: uint16
    segment: uint16
    user: ('<u2', (8,))
  user_int: ('<i4', (8,))
  user_float: ('<f4', (8,))
traj: object
data: object


In [ ]:
num_samp = {}

for acq in raw_data:
    key = int(acq['head']['number_of_samples'])
    num_samp[key] = num_samp.get(key, 0) + 1

# 128 for refscan, 768 for ksp
for key, val in num_samp.items():
    print(f"{key}: {val}")

128: 10000
768: 13500


In [20]:
ksp_data = [acq for acq in raw_data if acq['head']['number_of_samples'] == 768]
refscan_data = [acq for acq in raw_data if acq['head']['number_of_samples'] == 128]

In [22]:
from utils import build_kspace

kspace_main = build_kspace(ksp_data)
kspace_ref  = build_kspace(refscan_data)

In [23]:
print(kspace_main.shape)
print(kspace_ref.shape)

(768, 336, 256, 32)
(128, 100, 100, 32)
